In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from natsort import natsorted
import glob


In [2]:
sleep_scoring_df = pd.DataFrame()

pkl_filez = natsorted(glob.glob('/media/HlabShare/james/LFP_and_motion_pkls/*.pkl'))
for pklfile in pkl_filez:
    tmpdf = []
    tmpdf = pd.read_pickle(pklfile)

    sleep_scoring_df = pd.concat([sleep_scoring_df, tmpdf])

In [3]:
X = sleep_scoring_df[['delta', 'theta', 'alpha', 'beta', 'gamma', 'sigma', 'thetaratio']].values
Y = sleep_scoring_df[['sleep_labels']]

from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.data[index]
        y = self.labels[index]
        return x, y
    
ds = MyDataset(X, Y)
dl = DataLoader(ds, batch_size=64)

In [4]:
class Conv1DAutoencoder5Channels(nn.Module):
    def __init__(self):
        super(Conv1DAutoencoder5Channels, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv1d(5, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv1d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv1d(32, 64, kernel_size=3)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(64, 32, kernel_size=3),
            nn.ReLU(),
            nn.ConvTranspose1d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(16, 5, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
model = Conv1DAutoencoder5Channels()
criterion = nn.MSELoss()
optim = torch.optim.Adam(model.parameters())

In [5]:
model.train()
for ecpoch in range(10):
    for X, y in dl:
        print(X.shape)

KeyError: 0